In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# New call
from pipeline_newcall import generate_groups
import sys
import pickle, json
import numpy as np
from scipy.spatial.distance import cosine
from collections import Counter
sys.path.append("/home/ray__/ssd/ai-engine/pkg/")
import text_preprocessing.preprocess as tp
from extra_preprocess import preprocess_text

sys.path.append("/home/ray__/ssd/BERT/")
from gpt_feat_utils import GPT_Inference
#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu")
gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/ether/", device="cpu")

# S.E
#ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl", "rb"))
#com_map = pickle.load(open("e_data/com_map.pkl", "rb"))
#com_map_ranked = pickle.load(open("e_data/ranked_com.pkl","rb"))

# Ether
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether/ether_entity_feats_grp.pkl", "rb"))
com_map = pickle.load(open("e_data/ether/com_map.pkl", "rb"))
com_map_ranked = pickle.load(open("e_data/ether/ranked_com.pkl","rb"))

# gc = pickle.load(open("e_data/gc.pkl","rb"))
# lc = pickle.load(open("e_data/lc.pkl","rb"))

common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    ent_fv[ent] = ent_fv_full[ent]

def get_similar_entites(text_fv):
    similar_entities = []
    
    for ent, fv in ent_fv.items():
        similar_entities.append((ent, 1-cosine(text_fv, fv)))
    
    similar_entities = [ent for ent, score in sorted(similar_entities, key=lambda kv:kv[1], reverse=True)[:10]]
    return similar_entities

def get_ranked_groups(groups):
    groupid_segments = groups.items()
    group_ent = {}
    group_mapping = {}
    group_rank = {}
    group_rank_lc = {}
    group_rank_gc = {}
    for groupid, segments in groupid_segments:
        segments_agg_text = preprocess_text("".join(["".join(seg["originalText"] for seg in segments)]))
        segments_fv = []
        for seg in segments_agg_text:
            segments_fv.append(gpt_model.get_text_feats(seg))
        segments_fv = np.mean(segments_fv, axis=0)
        similar_entities = get_similar_entites(segments_fv)
        group_mapping[groupid] = [com_map[s] for s in similar_entities]
        group_ent[groupid] = similar_entities
        if Counter(group_mapping[groupid]).most_common()[0][0] in lc.keys() and lc[Counter(group_mapping[groupid]).most_common()[0][0]]!=[]:
            group_rank_lc[groupid] = sum(lc[Counter(group_mapping[groupid]).most_common()[0][0]])
        else:
            group_rank_gc[groupid] = gc[Counter(group_mapping[groupid]).most_common()[0][0]]
            
    lc_rank = sorted(group_rank_lc.items(), key=lambda kv:kv[1], reverse=True)
    gc_rank = sorted(group_rank_gc.items(), key=lambda kv:kv[1], reverse=True)

    index = 0
    for gid, freq in lc_rank:
        group_rank[gid] = index
        index +=1
    print ("lc and gc split: ",index)
    for gid, freq in gc_rank:
        group_rank[gid] = index
        index +=1

        #group_rank[groupid] = com_map_ranked[Counter(group_mapping[groupid]).most_common()[0][0]]
    
    sorted_groups = sorted(groupid_segments, key=lambda kv: group_rank[kv[0]])
    
    ranked_groups = {}
    for groupid, groupobj in sorted_groups:
        ranked_groups[groupid] = groupobj
        
    return ranked_groups, group_mapping, group_rank, group_ent

def update_gc_lc(ranked_groups, group_map):
    for groupid, groupobj in list(ranked_groups.items())[:5]:
        gc[Counter(group_map[groupid]).most_common()[0][0]] += Counter(group_map[groupid]).most_common()[0][1]
    lc_local = {}
    for groupid, groupobj in list(ranked_groups.items()):
        freq = Counter(group_map[groupid]).most_common()[0]
        if freq[0] in lc_local.keys():
            lc_local[freq[0]] += freq[1]
        else:
            lc_local[freq[0]] = freq[1]
    for freq in lc_local.items():
        if len(lc[freq[0]])<5:
            if lc[freq[0]]==[]:
                lc[freq[0]]=[freq[1]]
            else:
                lc[freq[0]].append(freq[1])
        else:
            lc[freq[0]].pop(0)
            lc[freq[0]].append(freq[1])
            
    pickle.dump(lc, open("e_data/ether/lc.pkl", "wb"))
    pickle.dump(gc, open("e_data/ether/gc.pkl", "wb"))
    return gc, lc

def compute_groups_new_call(req):
    groups = generate_groups(request)
    ranked_groups, group_map, group_rank, group_ent = get_ranked_groups(groups)
    #gc, lc = update_gc_lc(ranked_groups, group_map)
    gc = {}
    lc = {}
    return ranked_groups, group_map, group_rank, group_ent, gc, lc

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


{"level": "warning", "filename": "tokenization_openai.py", "lineno": 96, "module": "tokenization_openai", "ts": "2020-01-16T10:24:24.010053Z", "msg": "ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy."}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-01-16T10:24:24.294727Z", "msg": "Adding _start_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-01-16T10:24:24.299158Z", "msg": "Adding _delimiter_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-01-16T10:24:24.300053Z", "msg": "Adding _classify_ to the vocabulary"}


In [3]:
# file_set = ['enrichment_testset/podcast_k8s.txt', 'enrichment_testset/podcast_github.txt']
# test_set = ['topic_testing/sync_eng_2020_01_10.txt']
test_set = ['topic_testing/sync_eng_2020_01_10.txt']
test_flag = True
if not test_flag:
    for file in file_set:
        gc = pickle.load(open("e_data/ether/gc.pkl","rb"))
        lc = pickle.load(open("e_data/ether/lc.pkl","rb"))
        with open(file,'rb') as f:
            request = json.load(f)
            if isinstance(request, str):
                request = json.loads(request)

        ranked_groups, group_map, group_rank, group_ent, gc, lc = compute_groups_new_call(request)
else:
    for file in test_set:
        gc = pickle.load(open("e_data/ether/gc.pkl","rb"))
        lc = pickle.load(open("e_data/ether/lc.pkl","rb"))
        with open(file,'rb') as f:
            request = json.load(f)
            if isinstance(request, str):
                request = json.loads(request)

        ranked_groups, group_map, group_rank, group_ent, gc, lc = compute_groups_new_call(request)

Using  01DAAYHEKY5F4E02QVRJPTFTXV  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 183, "ts": "2020-01-16T10:26:06.725845Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-01-16T10:26:06.727957Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-01-16T10:26:15.783512Z", "msg": "Request Sent"}
('Hundred hundred add to my question was when you remember what he had to do handle the case of hanging a call when you are past or something like that.', '2020-01-10T06:01:04Z', '8d6db5f7d9b74c54ba38fe710ffcaf3f', '003af464-62b3-4566-afe0-97347acfa9bb') ('Diesel column so that the solvent should that so my question is did we hand we have to handle that case properly testing now.', '2020-01-10T06:01:24Z', '8d6db5f

('Bracket from from the domain wins every time when you channel is created size of currently.', '2020-01-10T06:17:10Z', 'b1e8787a9a1f4859ac11cbb6a8124fd9', 'afc7e967-708b-4699-b9e9-e53729dc905d') ('So right now what we are doing is like we are trying to have some specific set of files for each and every month for each and every channel basically.', '2020-01-10T06:16:20Z', 'fb52cb663aec4795aee38ccfd904d315', 'e59beda7-962a-4f34-bf78-b11e8c09fb53')
('It is just a copy when you when you associate it is a mind to a channel but what we have what we do moving forward is along with the domain mind.', '2020-01-10T06:17:10Z', 'b1e8787a9a1f4859ac11cbb6a8124fd9', 'afc7e967-708b-4699-b9e9-e53729dc905d') ('Like for example, today we have in our call this call.', '2020-01-10T06:18:24Z', '8d6db5f7d9b74c54ba38fe710ffcaf3f', 'c379d363-bdc1-45bc-aaaa-298ba8435f85')
('It is just a copy when you when you associate it is a mind to a channel but what we have what we do moving forward is along with the domai

{"level": "info", "filename": "grouper_segments.py", "lineno": 1039, "module": "grouper_segments", "edges before prunning": 705, "edges after prunning": 705, "modularity": 0.7386272738639507, "ts": "2020-01-16T10:26:20.161911Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
Hundred hundred add to my question was when you remember what he had to do handle the case of hanging a call when you are past or something like that.
Okay, okay, and for that are you know getting the proper even when you do start like when recording client stars and things like that?
All right, what else Arjun have not heard  margin in a long time?
Diesel column so that the solvent should that so my question is did we hand we have to handle that case properly testing now.
So maybe just go into a little more detail on where we are on the web and iOS.
So once this is done, then we can start to look at a couple of other things which is showing just handing the mole.
All right today, I guess the t

order difference: 0
Relevant sentence:  So and they were like that four digits for looking much better than this.    =====    Reverted to the four digits random four digits like the chance of those random is very less.
order difference: 1
Relevant sentence:  Reverted to the four digits random four digits like the chance of those random is very less.    =====    My point is I am okay with pollution the likelihood of it happening is very low it will be even higher because we think that we enter in the meeting names and the set of I was not working before because we used to create uuid is to join the car as a room ID and the source of the rating and they would use the uuid this unique nonrepeatable.
order difference: 0
Relevant sentence:  So there is no check most of the SQL queries with deleted is for projects that I have added yesterday.    =====    I have tested it in staging to have taken a dump refers to a into database and stood in my local is working as expected that that we can me

[(0, 0), (1, 1), (2, 2), (3, 3), (20, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 7), (17, 7), (9, 8), (10, 9), (24, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 14), (18, 15), (19, 16), (21, 17), (22, 18), (23, 19), (25, 20)]
[[["Yeah, that will be a little unfair because we will not know we might not know lot of key phrases from the first call. So we'll do that first and then I'm going to figure out what we can do to reduce that impact of aggressive filtration. "], '2020-01-10T06:10:16Z', 'b1e8787a9a1f4859ac11cbb6a8124fd9', '6b24fba4-7b04-478c-b11e-6647d424fd5f']] 




====================Group Cluster=========================
[[["Okay. Okay, so I have deployed and the like basically pause embolism through that circuit and it is working like quite well there was only like in first there was it was the makah waited almost a second after the lake cause started. So for the attic is adding a like reducing the first marker offset by one second and adding a 2 second setting and s

lc and gc split:  0


In [7]:
for pos, (groupid, groupobj) in enumerate(ranked_groups.items()):
    print ("\n\n\n Group Rank: ", pos ,"\n\n\n")
    print(*[g["originalText"] for g in groupobj], sep="\n\n")
    print("\n\n Similar Entities: ", group_ent[groupid])
    print ("\n mapped Entites :", group_map[groupid])
    #print ("\n mapped rank:", group_rank[groupid])




 Group Rank:  0 



So again, so it should work now right in the same direction is right. even though 

Leave him to call it the okay. Alright, so it looks like you tested it and it worked. 

Probably at least that's the team consensus. All right today, I guess the that the the other part is adding the form validation, but I guess we can add that in your when you give to get to it, right? 


 Similar Entities:  ['Pink', 'Link', 'Gypsy', 'Dixie', 'Janus', 'Ui', 'Sdk', 'C', 'Code', 'Hermes']

 mapped Entites : [42, 21, 5, 48, 5, 10, 48, 113, 80, 33]



 Group Rank:  1 



Okay. Okay, so I have deployed and the like basically pause embolism through that circuit and it is working like quite well there was only like in first there was it was the makah waited almost a second after the lake cause started. So for the attic is adding a like reducing the first marker offset by one second and adding a 2 second setting and so will verify that whether it's like seamless or not in past. 

Okay, b

In [30]:
for pos, (groupid, groupobj) in enumerate(ranked_groups.items()):
    print ("\n\n\n Group Rank: ", pos ,"\n\n")
    print(*[g["originalText"] for g in groupobj], sep="\n\n")
    print("\n\n Similar Entities: ", group_ent[groupid])
    print ("\n mapped Entites :", group_map[groupid])
    #print ("\n mapped rank:", group_rank[groupid])




 Group Rank:  0 


Is it is an open source tool that allows you to take advantage of on-premises hybrid or public Cloud infrastructure giving you the freedom to move workloads where ever you want it offers security networking and storage services and can manage more than one cluster at a time kubernetes makes more efficient use of hard work allowing you to maximize your resources and save money, but here's where things get tricky use a container orchestration tool like kubernetes. You describe the configuration of your application and yeah mol file this configuration. action file is where you tell kubernetes how to do things like gather container images how to establish networking between containers how to mount storage volumes in order to store logs for that container containers of deployed onto hosts usually replicated groups and when it's time to deploy a new container into a cluster kubernetes schedules the deployment and looks from the most appropriate host to place the contain